## Capstone project. Part 1. Retrieving data about neighborhoods in Toronto

In this part, you can find the code that helps to scrape necessary information about neighborhoods in Toronto from the Wikipedia page using the Beautiful Soup package. Then, you can discover how that data is wrangled and cleaned. Eventually, the data is represented in a pandas Dataframe.

In [136]:
# import working libraries and packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import csv
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium # map rendering library
import json
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Libraries imported')

Libraries imported


In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [2]:
# retrieve the text of code from the Wikipedia page
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"ba0629b5-5e28-48a0-b606-13f3ea4d08ec","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":951325562,"wgRevisionId":951325562,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toron

In [3]:
# extract table data and writing it to match variable
match = soup.find('tbody')
print(match.prettify())

<tbody>
 <tr>
  <th>
   Postal code
  </th>
  <th>
   Borough
  </th>
  <th>
   Neighborhood
  </th>
 </tr>
 <tr>
  <td>
   M1A
  </td>
  <td>
   Not assigned
  </td>
  <td>
  </td>
 </tr>
 <tr>
  <td>
   M2A
  </td>
  <td>
   Not assigned
  </td>
  <td>
  </td>
 </tr>
 <tr>
  <td>
   M3A
  </td>
  <td>
   North York
  </td>
  <td>
   Parkwoods
  </td>
 </tr>
 <tr>
  <td>
   M4A
  </td>
  <td>
   North York
  </td>
  <td>
   Victoria Village
  </td>
 </tr>
 <tr>
  <td>
   M5A
  </td>
  <td>
   Downtown Toronto
  </td>
  <td>
   Regent Park / Harbourfront
  </td>
 </tr>
 <tr>
  <td>
   M6A
  </td>
  <td>
   North York
  </td>
  <td>
   Lawrence Manor / Lawrence Heights
  </td>
 </tr>
 <tr>
  <td>
   M7A
  </td>
  <td>
   Downtown Toronto
  </td>
  <td>
   Queen's Park / Ontario Provincial Government
  </td>
 </tr>
 <tr>
  <td>
   M8A
  </td>
  <td>
   Not assigned
  </td>
  <td>
  </td>
 </tr>
 <tr>
  <td>
   M9A
  </td>
  <td>
   Etobicoke
  </td>
  <td>
   Islington Avenue
  </td>
 </

In [4]:
# write information to the csv file
csv_file = open('postalcodes.csv', 'w')
csv_writer = csv.writer(csv_file)

for line in match.find_all('tr'): # search through all tr blocks
    line_split = line.text.split('\n') # assigning a str type to a variable (using .text method) and splitting it by '\n' making a list

    pc = line_split[1] # choosing the second item from the list - postal code
    b = line_split[3] # choosing the fourth item from the list - borough
    n = line_split[5] # choosing the sixth item from the list - neighborhood
    
    csv_writer.writerow([pc, b, n]) # writing a a row with a postalcode, a borough and a neighborhood to the file

csv_file.close()

In [5]:
# read data from the csv file to the data frame
df = pd.read_csv('postalcodes.csv')
df = df.drop(np.where(df['Borough'] == 'Not assigned')[0]) # we don't need the rows with the 'Not assigned' boroughs
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [6]:
# replace '/' with ',' in the Neighborhood column
for i in range(df.shape[0]):
    df.iloc[i,2] = df.iloc[i,2].replace(' /', ',')
df.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [7]:
df.shape

(103, 3)

In [8]:
# sort values
df.sort_values('Postal code', ascending=True, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
9,M1B,Scarborough,"Malvern, Rouge"
18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
36,M1G,Scarborough,Woburn
45,M1H,Scarborough,Cedarbrae


In [9]:
# reset index
df.reset_index(inplace=True)
df.head()

,index,Postal code,Borough,Neighborhood
0,9,M1B,Scarborough,"Malvern, Rouge"
1,18,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,27,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,36,M1G,Scarborough,Woburn
4,45,M1H,Scarborough,Cedarbrae


In [10]:
# delete 'index' column
df.drop(['index'], axis=1, inplace=True)
df.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## Capstone project. Part 2. Adding geospatial coordinates

In this section, geospatial coordinates for every postal code in Toronto are read from the file to a pandas Dataframe and added to the Dataframe with infomation about neighborhoods in Toronto.

In [11]:
# read coordinates for postal codes from the file
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [12]:
# extract latitude and longitude
coord_lat = df_coord.iloc[:,1:2]
coord_long = df_coord.iloc[:,2:3]

In [13]:
df['Latitude'] = coord_lat
df.head()

,Postal code,Borough,Neighborhood,Latitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573
3,M1G,Scarborough,Woburn,43.770992
4,M1H,Scarborough,Cedarbrae,43.773136


In [14]:
df['Longitude'] = coord_long
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


#### We can also can create a dataset only with the Boroughs that containts 'Toronto' in their names

In [15]:
toronto_table=[]
for i in range(len(df['Borough'])):
    if 'Toronto' in df['Borough'][i]:
        toronto_table.append(df.loc[i])
df_toronto = pd.DataFrame(toronto_table).reset_index(drop=True)
print(df_toronto.shape)
df_toronto.head()

(39, 5)


,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


## Capstone project. Part 3. Exploring and clustering the neighborhoods in the city of Toronto

#### From this point, the following analysis will be based on the df_toronto dataset.

### Step 1. Map the data and explore one neighborhood

In [17]:
# define coordinates for the city of Toronto
address = 'Toronto'

geolocator = Nominatim(user_agent='my-application')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinates of the city of Toronto is {}, {}'.format(latitude,longitude))

The geographical coordinates of the city of Toronto is 43.6534817, -79.3839347


In [18]:
# create a map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude,longitude], zoom_start=12)

# add neighborhood markers on map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#9933ff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)

map_toronto

In [19]:
# define Foursquare credentials and version

CLIENT_ID = 'XTIAZWJMPTZRP5DKZF1V54VP4LSGZP0FIA0AOMUOEHUXZBQL'
CLIENT_SECRET = '4MHBGQHG5IXBVJEN21RYQ1AT5RKTWMZBHQC4JVZMBAC0TFN1'
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XTIAZWJMPTZRP5DKZF1V54VP4LSGZP0FIA0AOMUOEHUXZBQL
CLIENT_SECRET:4MHBGQHG5IXBVJEN21RYQ1AT5RKTWMZBHQC4JVZMBAC0TFN1


#### Let's explore the first neighborhood in our dataset

In [20]:
# get the name of neighborhood
df_toronto.loc[0, 'Neighborhood']

'The Beaches'

In [21]:
# get latitude and longitude coordinates of this neighborhood
n_lat = df_toronto.loc[0, 'Latitude']
n_lng = df_toronto.loc[0, 'Longitude']
n_name = df_toronto.loc[0, 'Neighborhood']

print('Latitude and longitude values of {} neighborhood are {}, {}'.format(n_name, n_lat, n_lng))

Latitude and longitude values of The Beaches neighborhood are 43.67635739999999, -79.2930312


#### Let's get top 25 venues that are in our neighborhood within a radius of 500 meters

In [22]:
# create the get request url
limit = 25
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    n_lat,
    n_lng,
    VERSION,
    radius,
    limit)
url

'https://api.foursquare.com/v2/venues/explore?client_id=XTIAZWJMPTZRP5DKZF1V54VP4LSGZP0FIA0AOMUOEHUXZBQL&client_secret=4MHBGQHG5IXBVJEN21RYQ1AT5RKTWMZBHQC4JVZMBAC0TFN1&ll=43.67635739999999,-79.2930312&v=20180605&radius=500&limit=25'

In [23]:
# make a request
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e9b3ace660a9f001b6d1536'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'The Beaches',
  'headerFullLocation': 'The Beaches, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 6,
  'suggestedBounds': {'ne': {'lat': 43.680857404499996,
    'lng': -79.28682091449052},
   'sw': {'lat': 43.67185739549999, 'lng': -79.29924148550948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bd461bc77b29c74a07d9282',
       'name': 'Glen Manor Ravine',
       'location': {'address': 'Glen Manor',
        'crossStreet': 'Queen St.',
        'lat': 43.67682094413784,
        'lng': -79.29394208780985,
        'labeledLatLngs': [{'labe

In [24]:
# define a function that extracts the category of the venue from the json file
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

# filter columns
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
nearby_venues = nearby_venues.loc[:,filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

#clean columns' names
nearby_venues.columns = [col.split('.')[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare'.format(nearby_venues.shape[0]))

nearby_venues

6 venues were returned by Foursquare


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869
4,Dip 'n Sip,Coffee Shop,43.678897,-79.297745
5,Seaspray Restaurant,Asian Restaurant,43.678888,-79.298167


#### Let's place all the venues on the map of the neighborhood

In [26]:
# create a map of Toronto using latitude and longitude values
map_neighborhood = folium.Map(location=[n_lat,n_lng], zoom_start=14)

# add neighborhood markers on map
for lat, lng, venue, category in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name'], nearby_venues['categories']):
    label = '{}, {}'.format(venue, category)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#9933ff',
        fill_opacity=0.7,
        parse_html=False).add_to(map_neighborhood)

map_neighborhood

### Step 2. Explore neighborhoods in Toronto

#### Let's create a function that repeats the same process as in Step 1 for all neighborhoods

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create an API request url
        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
            CLIENT_ID,
            CLIENT_SECRET,
            lat,
            lng,
            VERSION,
            radius,
            limit)
        
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # create rows for the neighborhood with information about each venue
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    # create a dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                            'Neighborhood Latitude',
                            'Neighborhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    
    return(nearby_venues)

#### Use the above function to find the venues for the Toronto neighborhoods using df_toronto dataset

In [28]:
toronto_venues = getNearbyVenues(df_toronto['Neighborhood'],df_toronto['Latitude'],df_toronto['Longitude'])
toronto_venues.head()

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High Park, The Junction South
Parkdale, Ron

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [29]:
toronto_venues.shape

(1602, 7)

Let's check how many venues were returned for each neighborhood

In [30]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,56,56,56,56,56,56
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,14,14,14,14,14,14
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",18,18,18,18,18,18
Central Bay Street,59,59,59,59,59,59
Christie,18,18,18,18,18,18
Church and Wellesley,73,73,73,73,73,73
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,34,34,34,34,34,34


Let's find out how many unique catgories of the venues we have in the toronto_venues dataset. 

In [31]:
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 228 unique categories


### Step 3. Analyze each neighborhood in Toronto

#### Let's implement one hot encoding to toronto_venues dataset

In [53]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')

# add neighborhood column back to dataframe
# as 'Neighborhood' is in the venue categories, let's call the column 'Neighborhood_' 
toronto_onehot['Neighborhood_'] = toronto_venues['Neighborhood']

# move Neighborhood_ column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood_,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,

Let's examine the new dataframe size

In [54]:
toronto_onehot.shape

(1602, 229)

#### Let's group rows by neighborhoods and by taking the mean of the frequency of occurence of each category

In [57]:
toronto_grouped = toronto_onehot.groupby('Neighborhood_').mean().reset_index()
toronto_grouped.head()

,Neighborhood_,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Cable Car,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Repair,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.017857,0.0,0.0,0.0,0.000000,0.017857,0.0,0.017857,0.035714,0.0,0.000000,0.0,0.017857,0.017857,0.0,0.035714,0.0,0.0,0.017857,0.000000,0.0,0.000000,0.0,0.017857,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.017857,0.0,0.035714,0.0,0.0,0.0,0.035714,0.0,0.0,0.0,0.000000,0.0,0.053571,0.053571,0.0,0.0,0.0,0.0,0.0,0.017857,0.000000,0.017857,0.000000,0.017857,0.0,0.017857,0.0,0.0,0.0,0.0,0.000000,0.0,0.017857,0.000000,0.0,0.0,0.0,0.000000,0.0,0.017857,0.0,0.0,0.0,0.000000,0.035714,0.000000,0.0,0.0,0.017857,0.0,0.0,0.0,0.0,0.0,0.017857,0.017857,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.017857,0.017857,0.000000,0.000000

Let's examine the new dataframe size

In [58]:
toronto_grouped.shape

(39, 229)

#### Let's print each neighborhood along with the top 5 most common venues 

In [90]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood_']:
    print('-----'+hood+'-----')
    temp = toronto_grouped[toronto_grouped['Neighborhood_']==hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq':5})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print()

-----Berczy Park-----
            venue     freq
0     Coffee Shop  0.05357
1    Cocktail Bar  0.05357
2      Restaurant  0.03571
3        Beer Bar  0.03571
4  Farmers Market  0.03571

-----Brockton, Parkdale Village, Exhibition Place-----
                   venue     freq
0                   Café  0.13043
1         Breakfast Spot  0.08696
2            Coffee Shop  0.08696
3     Italian Restaurant  0.04348
4  Performing Arts Venue  0.04348

-----Business reply mail Processing CentrE-----
                  venue     freq
0  Gym / Fitness Center  0.07143
1         Auto Workshop  0.07143
2         Garden Center  0.07143
3                Garden  0.07143
4    Light Rail Station  0.07143

-----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport-----
                 venue     freq
0      Airport Service  0.16667
1       Airport Lounge  0.11111
2     Airport Terminal  0.11111
3              Airport  0.05556
4  Rental Car Location  0.05556

#### Let's put the results into a pandas dataframe

First, let's write the function to sort the venues of one row in descending order 

In [101]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create a new dataframe and display the top 5 venues for each neighborhood

In [113]:
num_top_venues = 5

# create columns according to the number of the top venues (not more than 20)
indicators = ['st', 'nd', 'rd']
columns = ['Neighborhood']
for ind in range(num_top_venues):
    try:
        columns.append('{}{} common venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th common venue'.format(ind+1))

# create a new dataframe        
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood_']
for int in range(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[int, 1:] = return_most_common_venues(toronto_grouped.iloc[int, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st common venue,2nd common venue,3rd common venue,4th common venue,5th common venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Restaurant,Bakery
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Breakfast Spot,Gym,Bakery
2,Business reply mail Processing CentrE,Light Rail Station,Fast Food Restaurant,Brewery,Burrito Place,Auto Workshop
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Airport Lounge,Airport Terminal,Airport,Coffee Shop
4,Central Bay Street,Coffee Shop,Italian Restaurant,Sandwich Place,Café,Ice Cream Shop
5,Christie,Grocery Store,Café,Park,Gas Station,Italian Restaurant
6,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Restaurant,Yoga Studio
7,"Commerce Court, Victoria Hotel",Coffee Shop,Restaurant,Café,Hotel,Gym
8,Davisville,Dessert Shop,Sandwich Place,Gym,Coffee Shop,Pizza Place
9,Davisville North,Gym,Hotel,Breakfast Spot,Food & Drink Shop,Sandwich Place


### Step 4. Cluster neighborhoods

Let's run k-means to cluster neighborhoods into 3 clusters

In [152]:
# set number of clusters
num_clusters = 3

# drop the neighborhood column
toronto_grouped_clustering = toronto_grouped.drop('Neighborhood_', 1)

# run k-means clustering
kmeans = KMeans(n_clusters = num_clusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2])

Let's create a new dataframe that includes the cluster as well as the top 5 common venues for each neighborhood

In [153]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


# drop the column with the postal code in df_toronto
toronto_merged = df_toronto.drop('Postal code', 1)

# merge df_toronto with toronto_grouped to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st common venue,2nd common venue,3rd common venue,4th common venue,5th common venue
0,East Toronto,The Beaches,43.676357,-79.293031,2,Neighborhood,Pub,Coffee Shop,Health Food Store,Trail
1,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Frozen Yogurt Shop
2,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,2,Fast Food Restaurant,Gym,Pub,Liquor Store,Sandwich Place
3,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Brewery,Gastropub,Bakery
4,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Bus Line,Swim School,Farmers Market,Falafel Restaurant


Finally, let's visualize the resulting clusters

In [154]:
map_clusters = folium.Map(location=[latitude,longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(num_clusters)
ys = [i + x + (i*x)**2 for i in range(num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
for lat, lng, neigh, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(neigh) + ', Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
map_clusters

### Conclusion

Clustering and results visualizing show that Toronto neighborhoods have many features in common because Cluster 2 includes more than 90% of all neighborhoods considered. Thus, we could choose another clustering algorithm or to analyze all the neiborhoods in Toronto, not the only ones containing 'Toronto' word in their names.  